In [1]:
import OpenOPC
import time as t
import csv
import pandas as pd
import datetime
import pywintypes 
from datetime import datetime
from datetime import date
import os
import pyodbc
from pathlib import Path

pywintypes.datetime=pywintypes.TimeType

now = datetime.now()

today = date.today()
date = today.strftime("%d_%m_%Y")
extension = ".csv"
Filename = str(date) + extension

opc = OpenOPC.client()
servers = opc.servers()
opc.connect('CoDeSys.OPC.DA')
aliases = opc.list()
groups = opc.list('PLC')
salias = opc.list('PLC.Application')

maingroup = opc.list('PLC.Application.GVL.*')

ICR1_group = opc.list('PLC.Application.GVL.ICR1*')
ICR2_group = opc.list('PLC.Application.GVL.ICR2*')
INC_group = opc.list('PLC.Application.GVL.INC*')
MCR_group = opc.list('PLC.Application.GVL.MCR*')
OG_group = opc.list('PLC.Application.GVL.OG*')
PLC_group = opc.list('PLC.Application.GVL.PLC*')
A_group = opc.list('PLC.Application.GVL.A*')
R_group = opc.list('PLC.Application.GVL.R*')
S_group = opc.list('PLC.Application.GVL.S*')
T_group = opc.list('PLC.Application.GVL.T*')
WMS_group = opc.list('PLC.Application.GVL.W*')

iteration = 1
sql_it = 0
#Random_group = [ACDB_EM, AUX_EM, AUX_HT, RTCFORHmi, SPARE_EM, TVM_EM, WMS ]

Group_list = [ICR1_group, ICR2_group, INC_group, MCR_group,OG_group,A_group, R_group, S_group, T_group, WMS_group]


def isMultipleof5(n): 
    while ( n > 0 ): 
        n = n - 5
    if ( n == 0 ): 
        return 1
    return 0

In [3]:
def sql(z):
    # Read CSV File using Panda

    data = pd.read_csv (str(z) +"_"+str(date) + extension)
    df = pd.DataFrame(data, columns= ["Time", "Tag", "Value"])
    df.drop( df[ df['Value'] == 'False' ].index , inplace=True)
    df.drop( df[ df['Value'] == 'True' ].index , inplace=True)
    df = df.dropna(how='any',axis=0) 
    #print(df)
    
    df.to_csv(str(z) +"_"+str(date) +"Update"+ extension) 
    data1 = pd.read_csv (str(z) +"_"+str(date) +"Update"+ extension)   
    df1 = pd.DataFrame(data1, columns= ["Time", "Tag", "Value"])

    # Connect with SQL Server
    server = 'DESKTOP-4CIC0JV' 
    database = 'OPC' 
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database)
    cursor = conn.cursor()

    # Store Updated CSV into SQL Server 2016
    for index, row in df1.iterrows():  
        cursor.execute("INSERT INTO OPC.dbo.OPC_DATA (Time, Tag, Value) values(?,?,?)", row.Time, row.Tag, row.Value)

    conn.commit()
    os.remove(str(z) +"_"+str(date) +"Update"+ extension)
    


In [4]:
#-Group_list--------------------------------------------------------------------------------------------------------------------

while True:
    if ( isMultipleof5(iteration) == 1 ): 
        print("Go to SQL")
        sql(sql_it)
        sql_it = sql_it + 1
        iteration = iteration + 1
    #if ( isMultipleof5(iteration) == 0 ): 
    else: 
        print("Dont go to SQL")
        print("iteration" + str(iteration))
        filename = str(sql_it) +"_"+str(date) + extension 
        path = Path(filename)
        _csv_exist = path.exists()
        print("CSV:{1} {0}".format("Exist" if _csv_exist == True else "creating", filename))
        with open(filename, 'a', newline='') as file:
            writer = csv.writer(file)
            if _csv_exist == False:
                writer.writerow(["Time", "Tag", "Value"])
            for i in Group_list:
                for j in i:
                    for name, value, quality, time in opc.iread(opc.list('PLC.Application.GVL.'+j)):
                        writer.writerow([current_time,name,str(value)])
                t.sleep(1)
            iteration = iteration + 1
 
        

Dont go to SQL
iteration1
CSV:0_22_01_2021.csv creating
Dont go to SQL
iteration2
CSV:0_22_01_2021.csv Exist
Dont go to SQL
iteration3
CSV:0_22_01_2021.csv Exist
Dont go to SQL
iteration4
CSV:0_22_01_2021.csv Exist
Go to SQL
Dont go to SQL
iteration6
CSV:1_22_01_2021.csv creating
Dont go to SQL
iteration7
CSV:1_22_01_2021.csv Exist
Dont go to SQL
iteration8
CSV:1_22_01_2021.csv Exist
Dont go to SQL
iteration9
CSV:1_22_01_2021.csv Exist
Go to SQL
Dont go to SQL
iteration11
CSV:2_22_01_2021.csv creating
Dont go to SQL
iteration12
CSV:2_22_01_2021.csv Exist
Dont go to SQL
iteration13
CSV:2_22_01_2021.csv Exist


KeyboardInterrupt: 

In [ ]:
#opc.close()